## Getting YouTube captions

#### This notebook shows how to gather data from YouTube and create a dataframe with it.

In [85]:
#pip install youtube_transcript_api
# pip install google-api-python-client

In [124]:
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import datetime

In [125]:
api_key = API_KEY
# replace by the API key that you generate on Google Developers

video_id = VIDEO_ID
# replace by the video id which is the part of the URL after 'https://www.youtube.com/watch?v='



In [126]:
youtube = build('youtube', 'v3', developerKey= api_key)
#create a YouTube Data API v.3 object

In [127]:

srt = YouTubeTranscriptApi.get_transcript(video_id, languages = ['fr'])
# get the captions data
transcriptions = []
time_caption = []
for i in srt:
  transcriptions.append(i['text'])
  time = datetime.timedelta(seconds=round(i['start']))
  time_caption.append(time)

In [128]:
df = pd.DataFrame({
                 'time_caption' : time_caption,
                 'transcript': transcriptions })

time_captions = list(pd.DataFrame(df.resample('T', on='time_caption'))[0])
transcriptions = list(pd.DataFrame(df.resample('T', on= 'time_caption').agg({'transcript': ' '.join}))['transcript'])

In [129]:


request = youtube.videos().list(part= "contentDetails,snippet", id = video_id)
response = request.execute()
#make a request to the data of the video and execute it
#the resulting response is a dictionary with lots of metadata of the video
channel_id = [response['items'][0]['snippet']['channelId']] * len(transcriptions)
video_description = [response['items'][0]['snippet']['description']] * len(transcriptions)
upload_date = [response['items'][0]['snippet']['publishedAt'].replace('T', ' ').replace('Z', '')] * len(transcriptions)
channel_id = [channel_id] *len(transcriptions)
channel_title = [response['items'][0]['snippet']['channelTitle']]* len(transcriptions)
video_id = [response['items'][0]['id']]* len(transcriptions)
duration = [response['items'][0]['contentDetails']['duration'][2:].replace('M', ':').replace('S', '')] * len(transcriptions)
video_title = [response['items'][0]['snippet']['title']]* len(transcriptions)

In [131]:
df = pd.DataFrame({
                 
                 'channel_title': channel_title,
                 'channel_id': channel_id,
                 'video_title': video_title,
                 'video_id' : video_id,
                 'video_description': video_description,
                 'upload_date' : upload_date,
                 'duration': duration, 
                 'time_caption' : time_captions,
                 'transcript': transcriptions })
#create the dataframe

In [135]:
df.head()

,channel_title,channel_id,video_title,video_id,video_description,upload_date,duration,time_caption,transcript
0,Marine Le Pen,"[UCU3z3px1_RCqYBwrs8LJVWg, UCU3z3px1_RCqYBwrs8...",Discours de Marine Le Pen à Fréjus,KbvDa-sHATk,👉🏻 http://www.adhesions-rn.fr\n👉🏻 http://www.m...,2021-09-13 08:33:58,39:56,0 days 00:00:04,[Musique] [Musique] mes chers amis [Applaudiss...
1,Marine Le Pen,"[UCU3z3px1_RCqYBwrs8LJVWg, UCU3z3px1_RCqYBwrs8...",Discours de Marine Le Pen à Fréjus,KbvDa-sHATk,👉🏻 http://www.adhesions-rn.fr\n👉🏻 http://www.m...,2021-09-13 08:33:58,39:56,0 days 00:01:04,mes chers amis nous sommes un moment clé de no...
2,Marine Le Pen,"[UCU3z3px1_RCqYBwrs8LJVWg, UCU3z3px1_RCqYBwrs8...",Discours de Marine Le Pen à Fréjus,KbvDa-sHATk,👉🏻 http://www.adhesions-rn.fr\n👉🏻 http://www.m...,2021-09-13 08:33:58,39:56,0 days 00:02:04,mais aussi au sens littéral du terme en forme ...
3,Marine Le Pen,"[UCU3z3px1_RCqYBwrs8LJVWg, UCU3z3px1_RCqYBwrs8...",Discours de Marine Le Pen à Fréjus,KbvDa-sHATk,👉🏻 http://www.adhesions-rn.fr\n👉🏻 http://www.m...,2021-09-13 08:33:58,39:56,0 days 00:03:04,en 2022 cette élection présidentielle ça ne se...
4,Marine Le Pen,"[UCU3z3px1_RCqYBwrs8LJVWg, UCU3z3px1_RCqYBwrs8...",Discours de Marine Le Pen à Fréjus,KbvDa-sHATk,👉🏻 http://www.adhesions-rn.fr\n👉🏻 http://www.m...,2021-09-13 08:33:58,39:56,0 days 00:04:04,défenseurs de notre chère patrie que vous il n...
